In [13]:
import pandas as pd
import numpy as np
import os
import re
import subprocess as sp
import sys

In [16]:
os.makedirs('../data/cleaned', exist_ok=True)
os.makedirs('../data/graph', exist_ok=True)
os.makedirs('../data/edgelist', exist_ok=True)
os.makedirs('../data/circlize', exist_ok=True)

In [2]:
sixteenS_file = '../data/input/16S_AUV/NO-plastid-dada2-table.csv'
eighteenS_file = '../data/input/18SV9_AUV/dada2-table.csv'
meta_file = '../data/input/18SV9_AUV/230821_EH21_AUV_meta_with_dab.csv'
id_map_outfile = '../data/cleaned/AUV_id_map.tsv'
sixteenS_outfile = '../data/cleaned/AUV_sixteenS.tsv'
eighteenS_outfile = '../data/cleaned/AUV_eighteenS_v9.tsv'
meta_outfile = '../data/cleaned/AUV_metadata.tsv'

In [3]:
sixteenS = pd.read_csv(sixteenS_file)
eighteenS = pd.read_csv(eighteenS_file)

In [4]:
meta = pd.read_csv(meta_file).sort_values(by='sample-id').reset_index(drop=True)

In [5]:
to_drop = meta['sample-id'][meta['sample_type'].str.contains('control', case=False, na=True)].tolist()
to_drop = list(set(to_drop))

In [6]:
meta = meta[~meta['sample-id'].isin(to_drop)].drop('replicate', axis=1)

In [7]:
idxer = set(sixteenS.set_index('OTUID', drop=True).columns).intersection(
        set(eighteenS.set_index('OTUID', drop=True).columns)).intersection(
        set(meta['sample-id']))
idxer = list(idxer)
idxer.insert(0, 'OTUID')

In [8]:
sixteenS = sixteenS.loc[:,idxer]
eighteenS = eighteenS.loc[:,idxer]

In [9]:
idxer.remove('OTUID')
meta_filtered = meta[['sample-id', 'dabA_seriata_CPL', 'dabC_australis_CPL',
                      'dabA_australis_CPL', 'dabD_australis_sumof3_CPL']].set_index('sample-id', drop=True).loc[idxer]

In [10]:
def check_otus(df):# for each sample, compute Q3
    #  Each OTU must be above the non-zero Q3 in at least 5 sample
    q3s = []
    for col in df.columns:
        sample = df[col]
        q3 = sample[sample >= 0].describe()['25%']
        q3s.append(q3)
    # for each OTU, check if value > Q3 in at least 5 samples
    q3counts = []
    for i, row in df.iterrows():
        count = 0
        for j, val in enumerate(row):
            if val > q3s[j]:
                count += 1
        q3counts.append(count)
    finalcounts = pd.Series(q3counts, index=df.index)
    return finalcounts[finalcounts >= 5]
    
sixteenS_filtered = sixteenS.loc[check_otus(sixteenS.drop('OTUID', axis=1)).index.values]
eighteenS_filtered = eighteenS.loc[check_otus(eighteenS.drop('OTUID', axis=1)).index.values]
potus = pd.Series(sixteenS_filtered.index).apply(lambda x: 'potu_' + str(x)).values
eotus = pd.Series(eighteenS_filtered.index).apply(lambda x: 'eotu_' + str(x)).values
id_map = pd.DataFrame({'OTUID': sixteenS_filtered['OTUID'].values, 'short_id': potus})
id_map = pd.concat([id_map, pd.DataFrame({'OTUID': eighteenS_filtered['OTUID'].values,
                                          'short_id': eotus})])
id_map.to_csv(id_map_outfile, sep='\t', index=False)
sixteenS_filtered = sixteenS_filtered.drop('OTUID', axis=1).T
eighteenS_filtered = eighteenS_filtered.drop('OTUID', axis=1).T
sixteenS_filtered.columns = potus
eighteenS_filtered.columns = eotus
sixteenS_filtered.to_csv(sixteenS_outfile, sep='\t')
eighteenS_filtered.to_csv(eighteenS_outfile, sep='\t')
meta_filtered.to_csv(meta_outfile, sep='\t')

In [11]:
sixteen_cols = pd.read_csv(sixteenS_outfile, index_col=0, sep='\t').columns
eighteen_cols = pd.read_csv(eighteenS_outfile, index_col=0, sep='\t').columns
len(sixteen_cols) + len(eighteen_cols)

1718

## Run FlashWeave

In [12]:
sp.run(['julia', f'src/run_flashweave_v9.jl',
        'data/cleaned/AUV_sixteenS.tsv',
        'data/cleaned/AUV_eighteenS_v9.tsv',
        'data/cleaned/AUV_metadata.tsv',
        'data/graph/AUV_network_output_v9.gml'], cwd='..')


### Loading data ###

### Normalizing ###

Normalization
	-> multiple data sets provided, using separate normalization mode

### Learning interactions ###

Inferring network with FlashWeave - sensitive (conditional)

	Run information:
	sensitive - true
	heterogeneous - false
	max_k - 3
	alpha - 0.01
	sparse - false
	workers - 1
	OTUs - 1718
	MVs - 0

Automatically setting 'n_obs_min' to 20 for enhanced reliability
Computing univariate associations

Univariate degree stats:
Summary Stats:
Length:         1718
Missing Count:  0
Mean:           23.717113
Minimum:        0.000000
1st Quartile:   3.000000
Median:         18.000000
3rd Quartile:   36.000000
Maximum:        104.000000



Starting conditioning search

Preparing workers..

Done. Starting inference..
Starting convergence checks at 1490 edges.
Latest convergence step change: 1.30749

Postprocessing
Complete

Finished inference. Total time taken: 6.356s


CompletedProcess(args=['julia', 'src/run_flashweave_v9.jl', 'data/cleaned/AUV_sixteenS.tsv', 'data/cleaned/AUV_eighteenS_v9.tsv', 'data/cleaned/AUV_metadata.tsv', 'data/graph/AUV_network_output_v9.gml'], returncode=0)